# Many-body simulations - $S^{eff}(q)$ with 12-beads model

# Import packages

In [ ]:
import numpy as np
from jinja2 import Template
import os
import matplotlib.pyplot as plt
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 100%; }
    div#menubar-container     { width: 100%; }
    div#maintoolbar-container { width: 100%; }
</style>
"""))

plt.rcParams.update({'font.size':16,'legend.frameon':True,'figure.figsize':[12,8],'xtick.major.size':7,'ytick.major.size':7,'legend.labelspacing':1})

# Go and write down the working directory

In [ ]:
try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd -q $workdir

%cd $workdir

# Functions

In [ ]:
# Length of the cube side in Å corresponding to some protein concentratiom in g/l
def Length_in_gl(cp, N_of_proteins, Mw):
    return np.cbrt(N_of_proteins/(cp*(1/Mw)*6.022e23*1e-27)) 

# Length of the cube in Å corresponding to some protein concentratiom in mol/dm^3
def Length_in_M(cp, N_of_proteins ):
    return np.cbrt(N_of_proteins/(cp*6.022e23*1e-27)) 

# Look at the simulation time
def Sim_time(jsonfile):
    with open(jsonfile) as f:
        d = json.load(f) # --> dict
        return d["simulation time"]['in seconds']

# Create 12 bead model in pqr format 
def createpqr_12b(filename, sigma):
    beads = 12
    f = open(filename,'w')
    f.write(str(beads)+'\n')
    for i in range(beads):
        #############################################################################################
        if i ==0:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , 0, -sigma, 0, 0, sigma/2))        
        if i ==1:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , 0, 0, 0, 0, sigma/2))
        if i ==2:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , 0, sigma, 0, 0, sigma/2))            
        if i ==3:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , 0, 2*sigma, 0, 0, sigma/2))
        #####################################################################################################################
        if i ==4:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , sigma*np.cos(np.pi/3), 2*sigma+sigma*np.sin(np.pi/3), 0, 0, sigma/2))
        if i ==5:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , -sigma*np.cos(np.pi/3), 2*sigma+sigma*np.sin(np.pi/3), 0, 0, sigma/2))
        #####################################################################################################################
        if i ==6:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , 2*sigma*np.cos(np.pi/3), 2*sigma+2*sigma*np.sin(np.pi/3), 0, 0, sigma/2))

        if i ==7:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , -2*sigma*np.cos(np.pi/3), 2*sigma+2*sigma*np.sin(np.pi/3), 0, 0, sigma/2))
        ######################################################################################################################
        if i ==8:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , 3*sigma*np.cos(np.pi/3), 2*sigma+3*sigma*np.sin(np.pi/3), 0, 0, sigma/2))

        if i ==9:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , -3*sigma*np.cos(np.pi/3), 2*sigma+3*sigma*np.sin(np.pi/3), 0, 0, sigma/2))
        ######################################################################################################################
        if i ==10:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , 4*sigma*np.cos(np.pi/3), 2*sigma+4*sigma*np.sin(np.pi/3), 0, 0, sigma/2))

        if i ==11:
            f.write('{0:6} {1:4} {2:4} {3:4} {4:5} {5:11.3f} {6:8.3f} {7:8.3f} {8:8.3f} {9:6.3f}\n'\
                .format('ATOM', i, 'BEAD', 'A', 0 , -4*sigma*np.cos(np.pi/3), 2*sigma+4*sigma*np.sin(np.pi/3), 0, 0, sigma/2))
        ######################################################################################################################
       
    
    f.write('END'+'\n')        
    f.close()

# Create simulation input in yml format
def create_input(macro, micro, nstep, nskip, L, N, sigma, q, Dl, eps, p):
    with open('input.yml', 'w') as input_file:
        d = input_file.write(Input.render(macro = macro,
                                          micro = micro,
                                          nstep = nstep,
                                          nskip = nskip,
                                              L = L,
                                              N = N,
                                          sigma = sigma,
                                              q = q,
                                             Dl = Dl,
                                            eps = eps,
                                              p = p
                                          
                                         ))
        return d 

def kappa(conc, Z, rhob, rhos, sigma_HS):#sigma in nm
    sigma = sigma_HS
    lB = 0.714 #nm
    Na = 6.02214076e23 # 1/mol
    rho = 4.068e15 #particles per ml
    phi = conc*rho*1e-21 * (np.pi* sigma**3) / 6
    PHI = 1 / (1 - phi)
    k =    4 * np.pi * lB * ( PHI * conc*Z*rho*1e-21 + 2*rhob*Na*1e-27 +2*rhos*Na*1e-27  ) 
    return np.sqrt(k)

def Rg_pqr(Input):
    coordinates = np.loadtxt(Input, unpack=True, usecols=(5,6,7), skiprows=1, max_rows=9)
    
    N = len(coordinates[0])
    
    cm_x = 0
    cm_y = 0
    cm_z = 0
   

    for i in range(len(coordinates[0])):
        #print(cm_x)
        cm_x = cm_x + coordinates[0][i] 
        cm_y = cm_y + coordinates[1][i] 
        cm_z = cm_z + coordinates[2][i]

    cm = cm_x/N, cm_y/N, cm_z/N
    #print('c.o.m.: ',cm[0],cm[1],cm[2])
    
    rg_x = 0
    rg_y = 0
    rg_z = 0

    for i in range(len(coordinates[0])):
        rg_x = rg_x + ( coordinates[0][i] - cm[0] )**2
        rg_y = rg_y + ( coordinates[1][i] - cm[1] )**2
        rg_z = rg_z + ( coordinates[2][i] - cm[2] )**2    

    Rg = np.sqrt( ( rg_x + rg_y + rg_z ) / N )  
    
    return print('Rg : ', round(Rg,1), 'Å')

# Choose sigma bead to match the radius of gyration

In [ ]:
sigma_values = [21, 19.6]

for s in sigma_values:
    createpqr_12b('12b_sigma_'+str(s)+'.pqr', s)
    Rg_pqr('12b_sigma_'+str(s)+'.pqr')

# Runs - HS and Yukawa with soft attraction potentials

In [ ]:
# Go and write down the working directory

try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd -q $workdir

%cd $workdir
####################

Input = Template("""
temperature: 298.15
geometry: {type: cuboid, length: [{{L}},{{L}},{{L}}]}
mcloop: {micro: {{micro}}, macro: {{macro}}}
random: {seed: hardware}
energy:
    - nonbonded:
        cutoff_g2g: {{9*sigma+8*Dl}} 
        default:
            - hardsphere:
                mixing: arithmetic 
            - custom:
                function: lB * q^2 * ( exp( sigma / (2 * Dl) ) / ( 1 + sigma / (2 * Dl) ) )^2 * ( exp(-r / Dl) / r ) - eps * ( sigma / r )^6       
                constants:
                    lB: 7.14
                    q: {{q}}
                    Dl: {{Dl}}
                    sigma: {{sigma}}
                    eps: {{eps}}                    
                    
atomlist:
    - BEAD:   { sigma: {{sigma}}}      
moleculelist:
    - 12_beads:
        structure: input.pqr
        insdir: [1,1,1]
        insoffset: [0,0,0]
        keepcharges: False
        keeppos: False
        rigid: True

insertmolecules:
    - 12_beads: {N: {{N}}, inactive: False }

moves:
    - moltransrot: {molecule: 12_beads, dir: [1,1,1], dprot: 0.3, dp: 100 }

analysis:
#    - sanity: {nstep: {{nstep}} }
#    - systemenergy: {file: energy.dat, nstep: {{nstep}}, nskip: {{nskip}}}
#    - molecule_density: {nstep: {{nstep}}}
    - savestate: {file: state.json}
    - savestate: {file: confout.pqr}
#    - xtcfile: {file: traj.xtc, nstep: {{nstep}}, nskip: {{nskip}} }
#    - molrdf: { file: rdf.dat, nstep: 1, dr: 1, name1: 12_beads, name2: 12_beads, dim: 3 }
    - scatter: {file: debye.dat, nstep: 1, molecules: ["12_beads"], com: False, scheme: explicit, pmax: {{p}}}
#    - scatter: {file: debye_COM.dat, nstep: 1, molecules: ["12_beads"], com: True, scheme: explicit, pmax: {{p}}}
""")
#######################################################################################

####### Conditions ########
Mw = 148000               #
N = 500                  #
########################################
potential = 'HS_Yukawa_LJ'                       
c_range = [  20, 50, 100, 150, 200 ]                    
#eps_range = [ 0, 0.2, 0.3, 0.4, 0.5, 0.6 ] # V4
eps_range = [ 0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6 ] # LJ
I = [ 7 ]                                         
Z = [ 22, 24 ]             
#########################################
micro = 1      #                                                                                     
macro = 10000  #
nstep = 1000   #
nskip = 0      #
################
   
### Runs ###
%mkdir runs
%cd runs
%mkdir $potential
%cd $potential

for i in I:
    %mkdir $i
    %cd -q $i
    for c in c_range:
        %mkdir $c
        %cd -q $c
        for z in Z:
            %mkdir $z
            %cd -q $z
            for eps in eps_range:
                %mkdir $eps
                %cd $eps
                
                if i ==7:
                    sigma = 21
                    sigma_HS = 96.4
                if i ==57:
                    sigma = 19.6
                    sigma_HS = 90

                L = Length_in_gl(c, N, Mw)
                Dl = 10/kappa(c, z, i, 0, sigma_HS/10 )
                createpqr_12b('input.pqr', sigma )
                create_input(macro, micro, nstep, nskip, L, N, sigma, z/12, Dl, eps, 25)

                exists = os.path.isfile('state.json')
                if exists:
                    # Path for Faunus executable
                    !export OMP_NUM_THREADS=1;yason.py Input.yml | faunus -v2 --state state.json #> out
                else:
                    # Path for Faunus executable
                    !export OMP_NUM_THREADS=1;yason.py Input.yml | faunus -v 2  #> out 
                           
                %cd -q ../ 
            %cd -q ../
        %cd -q ../
    %cd -q ../ 

%cd -q ../../